In [ ]:
import sys
from utils import *

sys.setrecursionlimit(100000000)

def cm_factor(N, D, B=32, debug=True):
    """
    Implements a simplified version of Cheng's 4p - 1 elliptic curve complex multiplication based factorization algorithm.
    Targets moduli where one (or many) prime factors satisfies the equality 4p_i - 1 = D * s^2, where D is a squarefree integer
    and s is a randomly generated one between an upper and lower bound.
    
    Original paper title: I want to break square-free: The 4p - 1 factorization method and its RSA backdoor viability 
    Link: https://crocs.fi.muni.cz/_media/public/papers/2019-secrypt-sedlacek.pdf
    
    :param N: integer to be factored
    :param D: squarefree integer satisfying 4p - 1 = D * s^2
    :param B: number of iterations to run the algorithm for
    :param debug: switches debugging information on/off
    :return: a tuple corresponding to p, q, or failure (-1)
    """

    # If D is not squarefree then we terminate immediately.
    assert D.is_squarefree(), "D must be squarefree."

    # Computes the -Dth Hilbert polynomial modulo N and quotient ring Q = Z_N[x] / <H_{-D, N}>

    Z_N = Zmod(N)
    P = Z_N[x]
    H = P(hilbert_class_polynomial(-D))
    Q = P.quotient_ring(H)

    # j is the equivalence class corresponding to [X] in Q.
    j = Q(x)

    # The paper claims that we can treat both 1728 - j and H as polynomials in Z_N[X] and calculate the inverse of
    # 1728 - j and H via egcd. This doesn't quite work off the shelves, so we instead accomplish this by treating 
    # 1728 - j as an element of Q = Z_N[x] / <H_{-D, n}> and lifting it back into the base ring.  
    # Sage implements this via the .lift() method.

    if debug:
        print("Calculating inverse mod of 1728 - j with H...")

    try:
        k = j * polynomial_inv_mod((1728 - j).lift(), H)
    except ValueError as e:
        r = gcd(int(e.args[1].lc()), N)
        return int(r), int(N // r)

    # Constructs an elliptic curve described by the equation y^2 = x^3 + ax + b where a = 3k and b = 2k over Q.
    # This is so we can calculate the division polynomial psi_n(a, b, x_i) later.
    E = EllipticCurve(Q, [3*k, 2*k])

    for i in range(B):

        # Obtains a random element from Z_N for the division polynomial.
        x_i = Z_N.random_element()

        if debug:
            print("Calculating division polynomial with x_i...")

        # Calculates the division polynomial modulo n: psi_n(a, b, x_i)
        z = E.division_polynomial(N, x=Q(x_i))

        # If our egcd implementation throws an exception, then we know that the r polynomial during the computation
        # has no inverse; we return this polynomial in our exception and then take the gcd of its leading coefficient with N.
        # Otherwise we take the gcd of d and N normally.
        try:
            d, _, _ = polynomial_egcd(z.lift(), H)
        except ValueError as e:
            r = gcd(int(e.args[1].lc()), N)
            return int(r), int(N // r)

        r = gcd(int(d), N)

        # Ensure that the factorization is nontrivial: i.e. r != 1 and r != N
        if 1 < r < N:
            return r, N // r
    
    return -1

if __name__ == "__main__":
    D = 427
    N = 404346648884597392605186375266254608649029175749025041311837330226523951380438816899580202048378908742814874153913081850066981021681949387875292053626788162948768889577117691921764786649412518087820362752090144069799045650706666541753120112708078914947299130801107134679432427667077893195345985056151062828188286901067724903830851410360396781111355290705021857613616717153933295263748265273164410819614021366920649837245670830338768555276561832467231485276869038117956933374984985391439855351449732597447804319032602280378261379721623867107254208096171778176518716445122699073475371662726449266915661919213790336664037723748275174633678518215591678617518596185895050893155749090148764342061102700263817455924583481055968619397624336871688495386566620206402698583647373521356152369317757009864876220991658527727687091920103260006624596861448704791961989352239282402333353751615431119565492068541681690066537776509460887031172075008361291074011597869706212760205400756854397344460880422800109964921113312876784750556152575069062299599305531272488454700401123289608535234753660854032419603223442797680605371354832525209828296329631613909574938905108585974691864490395385545086757093590596299139431863539103513806451598705641113210996323741276674714596960638509198166737848661721418002037773417867188268740270930169315142845925308755907322282700061823678558777025896252709719281538126072212270217861897296387535436208198633419038655256868764842922566790109305353071860437495907787471890392990733885985345069948273829613023679827159075446520212863573178949020931438432067035452016683867158983328997529082261162959770260423931085686734064286368639863494845591021903776036861931271649104146718210847571234532059095631058979940977752950022688882413664773583165622687611209696750371602428089124969102738112146929576825130445265527821371219399545644671814366439
    print("Factoring...")
    r, s = cm_factor(N, D)
    assert r * s == N
    print(f"Factorization successful! N = {r} * {s}")